## 1. Instalação dos pacotes necessários

In [79]:
# %pip install requests
# %pip install beautifulsoup4
# %pip install pandas
# %pip install numpy
# %pip install scikit-learn
# %pip install plotly

## 2. Script do WebScrapping

In [80]:
# import requests
# import time
# import datetime
# import os
# import json
# import re
# import pprint

# from bs4 import BeautifulSoup
# from urllib.parse import urlparse

# headers = {'User-Agent': 'Mozilla/5.0'}
    
# def get_price(url):
#     response = requests.get(url, headers=headers)
#     soup = BeautifulSoup(response.text, 'html.parser')
    
#     for svg in soup.find_all('svg'):
#         svg.extract()
        
#     product_info = []
    
#     cards = soup.find_all('article', {'class': 'productCard'})
    
#     for card in cards:
#         review_div = card.find('div', {'class': 'labelTotalAvaliacoes'})
#         if review_div:
#             review = review_div.text.strip()
#         full_name = card.find('span', {'class': 'nameCard'}).text.strip()
#         product_link = card.find('a', {'class': 'productLink'})['href']
        
#         product_info.append({ 'full_name': full_name,
#                              'link': product_link })
    
#     return product_info

# def get_product_info(url, product):
#     url = f"{url}{product['link']}"
#     response = requests.get(url, headers=headers)
#     soup = BeautifulSoup(response.text, 'html.parser')

#     script = soup.find('script', {'type': 'application/ld+json'})
#     if script:
#         script_text = script.text
#         json_data = json.loads(script_text)
#         sku = json_data.get('sku')
#         brand_name = json_data.get('brand', {}).get('name')

    
#     script_nd = soup.find('script', {'type': 'application/json', 'id': '__NEXT_DATA__'})
#     if script_nd:
#         script_nd_text = script_nd.text
#         json_data_nd = json.loads(script_nd_text)
#         json_initial = json_data_nd.get('props').get('pageProps').get('initialZustandState').get('descriptionProduct')
#         json_catalog_product = json_data_nd.get('props').get('pageProps').get('initialZustandState').get('catalogProduct')
        
#         weight = json_catalog_product.get('weight')
#         price = json_catalog_product.get('price')
#         primePrice = json_catalog_product.get('primePrice')
#         primePriceWithDiscount = json_catalog_product.get('primePriceWithDiscount')
#         oldPrice = json_catalog_product.get('oldPrice')
#         oldPrimePrice = json_catalog_product.get('oldPrimePrice')
#         priceWithDiscount = json_catalog_product.get('priceWithDiscount')
#         discountPercentage = json_catalog_product.get('discountPercentage')
#         rating = json_catalog_product.get('rating')
#         ratingCount = json_catalog_product.get('ratingCount')
#         available = json_catalog_product.get('available')
#         warranty = json_initial.get('warrantyString')
    
#     product_info = { 'sku': sku,
#                     'brand_name': brand_name,
#                     'weight': weight,
#                     'price': price,
#                     'primePrice': primePrice,
#                     'primePriceWithDiscount': primePriceWithDiscount,
#                     'oldPrice': oldPrice,
#                     'oldPrimePrice': oldPrimePrice,
#                     'priceWithDiscount': priceWithDiscount,
#                     'discountPercentage': discountPercentage,
#                     'rating': rating,
#                     'ratingCount': ratingCount,
#                     'available': available,
#                     'warranty': warranty}
    
#     return product_info

# def get_dominio(url):
#     parsed_url = urlparse(url)
#     return parsed_url.netloc

# def save_to_file(data):
#     file_exists = False
#     if os.path.isfile("scraped_data.csv"):
#         file_exists = True
    
#     with open("scraped_data.csv", "a") as file:
#         if not file_exists:
#             keys = data[0].keys()
#             print(';'.join(keys))
#             file.write(f"{';'.join(keys)};date\n")
            
#         for item in data:
#             file.write(f"{item['full_name']};{item['link']};{item['sku']};{item['brand_name']};{item['weight']};
#                        {item['price']};{item['primePrice']};{item['primePriceWithDiscount']};{item['oldPrice']};
#                        {item['oldPrimePrice']};{item['priceWithDiscount']};{item['discountPercentage']};{item['rating']};
#                        {item['ratingCount']};{item['available']};{item['warranty']};{datetime.datetime.now()}\n")

# def main():
#     pesquisa = 'rtx 4070 ti'
#     url = f"https://www.kabum.com.br/busca/{pesquisa}"
    
#     intervalo = 60 * 10 # 10 minutos
#     contador = 0
    
#     # while True:
#     try:
#         products = get_price(url)
#         products_info = []
#         for product in products:
#             info = get_product_info(f'https://{get_dominio(url)}/', product)
#             products_info.append(info)
        
#         list_products = []
#         if len(products) == len(products_info):
#             for p, pi in zip(products, products_info):
#                 list_products.append({  'full_name': p['full_name'],
#                                         'link': p['link'],
#                                         'sku': pi['sku'],
#                                         'weight': pi['weight'],
#                                         'price': pi['price'],
#                                         'primePrice': pi['primePrice'],
#                                         'primePriceWithDiscount': pi['primePriceWithDiscount'],
#                                         'oldPrice': pi['oldPrice'],
#                                         'oldPrimePrice': pi['oldPrimePrice'],
#                                         'priceWithDiscount': pi['priceWithDiscount'],
#                                         'discountPercentage': pi['discountPercentage'],
#                                         'rating': pi['rating'],
#                                         'ratingCount': pi['ratingCount'],
#                                         'available': pi['available'],
#                                         'warranty': pi['warranty']
#                                     })
        
#         # print(json.dumps(products[0], indent=4))
#         # print(json.dumps(products_info[0], indent=4))
#         # print(json.dumps(list_products, indent=4))
        
#         save_to_file(list_products)
#     except Exception as e:
#         print(f"Erro ao obter preços dos produtos na Kabum: {e}")
#     contador += 1
#         # time.sleep(intervalo)

# if __name__ == "__main__":
#     main()

## 3. Tratamento dos dados

In [81]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/scraped_data.csv", sep=";", encoding='latin1')
df.head(10)

,full_name,model,link,sku,brand_name,weight,price,primePrice,primePriceWithDiscount,oldPrice,oldPrimePrice,priceWithDiscount,discountPercentage,rating,ratingCount,available,warranty,date
0,placa-de-video-rtx-4070-ti-ventus-msi-nvidia-g...,rtx 4070 ti,https://www.kabum.com.br//produto/403373/placa...,403373,MSI,1732,6117.64,5934.11,5043.99,6117.64,6117.64,5199.99,15,4,8,True,36 meses de garantia,2024-04-25 03:36:29.337716
1,placa-de-video-rtx-4070-ti-super-gigabyte-gami...,rtx 4070 ti,https://www.kabum.com.br//produto/517737/placa...,517737,Gigabyte,1816,6941.16,0.00,0.00,0.00,0.00,5899.99,15,0,0,True,36 meses de garantia,2024-04-25 03:36:29.337716
2,placa-de-video-rtx-4070-ti-super-msi-16g-ventu...,rtx 4070 ti,https://www.kabum.com.br//produto/520532/placa...,520532,MSI,1198,6352.93,0.00,0.00,0.00,0.00,5399.99,15,5,2,True,12 meses de garantia,2024-04-25 03:36:29.337716
3,placa-de-video-rtx-4070-ti-gaming-x-slim-12g-m...,rtx 4070 ti,https://www.kabum.com.br//produto/495568/placa...,495568,MSI,1890,6117.64,5934.11,5043.99,6117.64,6117.64,5199.99,15,5,9,True,12 meses de garantia,2024-04-25 03:36:29.337716
4,placa-de-video-rtx-4070-ti-super-gaming-verto-...,rtx 4070 ti,https://www.kabum.com.br//produto/520758/placa...,520758,Pny,1570,5199.99,0.00,0.00,0.00,0.00,5199.99,0,0,0,True,36 meses de garantia,2024-04-25 03:36:29.337716
5,placa-de-video-rtx-4070-ti-super-sg-galax-nvid...,rtx 4070 ti,https://www.kabum.com.br//produto/518725/placa...,518725,Galax,5735,7176.46,6823.52,5799.99,7176.46,7176.46,6099.99,15,5,2,True,12 meses de garantia,2024-04-25 03:36:29.337716
6,placa-de-video-rtx-4070-ti-ventus-3x-trio-oc-m...,rtx 4070 ti,https://www.kabum.com.br//produto/418040/placa...,418040,MSI,1730,6601.06,0.00,0.00,0.00,0.00,6601.06,0,0,0,True,NaN,2024-04-25 03:36:29.337716
7,placa-de-video-rtx-4070-ti-super-gigabyte-aero...,rtx 4070 ti,https://www.kabum.com.br//produto/517736/placa...,517736,Gigabyte,1896,6941.16,6732.93,5722.99,6941.16,6941.16,5899.99,15,0,0,True,36 meses de garantia,2024-04-25 03:36:29.337716
8,placa-de-video-rtx-4070-ti-rog-strix-gaming-as...,rtx 4070 ti,https://www.kabum.com.br//produto/403370/placa...,403370,ASUS,2640,7999.99,7759.99,6595.99,7999.99,7999.99,6799.99,15,5,4,True,36 meses de garantia,2024-04-25 03:36:29.337716
9,placa-de-video-rtx-4070-ti-super-msi-16g-gamin...,rtx 4070 ti,https://www.kabum.com.br//produto/520529/placa...,520529,MSI,1800,7411.75,7189.40,6110.99,7411.75,7411.75,6299.99,15,5,2,True,12 meses de garantia,2024-04-25 03:36:29.337716


In [82]:
print(df.dtypes)

full_name                  object
model                      object
link                       object
sku                         int64
brand_name                 object
weight                      int64
price                     float64
primePrice                float64
primePriceWithDiscount    float64
oldPrice                  float64
oldPrimePrice             float64
priceWithDiscount         float64
discountPercentage          int64
rating                      int64
ratingCount                 int64
available                    bool
warranty                   object
date                       object
dtype: object


In [32]:
import numpy as np
import re

def extrair_numeros(texto):
    if pd.isna(texto):
        return 0
    else:
        numeros = re.findall(r'\d+', texto)
        if numeros:
            return int(numeros[0])
        else:
            return 0

In [83]:
df2 = df

df2['full_name'] = df2['full_name'].astype(str).str.strip()
df2['available'] = df['available'].astype(int)
df2['warranty'] = df2['warranty'].apply(extrair_numeros)
df2['date'] = pd.to_datetime(df2['date'])
df2['model'] = df2['model'].astype(str).str.upper()

In [84]:
df2.head(5)

,full_name,model,link,sku,brand_name,weight,price,primePrice,primePriceWithDiscount,oldPrice,oldPrimePrice,priceWithDiscount,discountPercentage,rating,ratingCount,available,warranty,date
0,placa-de-video-rtx-4070-ti-ventus-msi-nvidia-g...,RTX 4070 TI,https://www.kabum.com.br//produto/403373/placa...,403373,MSI,1732,6117.64,5934.11,5043.99,6117.64,6117.64,5199.99,15,4,8,1,36,2024-04-25 03:36:29.337716
1,placa-de-video-rtx-4070-ti-super-gigabyte-gami...,RTX 4070 TI,https://www.kabum.com.br//produto/517737/placa...,517737,Gigabyte,1816,6941.16,0.00,0.00,0.00,0.00,5899.99,15,0,0,1,36,2024-04-25 03:36:29.337716
2,placa-de-video-rtx-4070-ti-super-msi-16g-ventu...,RTX 4070 TI,https://www.kabum.com.br//produto/520532/placa...,520532,MSI,1198,6352.93,0.00,0.00,0.00,0.00,5399.99,15,5,2,1,12,2024-04-25 03:36:29.337716
3,placa-de-video-rtx-4070-ti-gaming-x-slim-12g-m...,RTX 4070 TI,https://www.kabum.com.br//produto/495568/placa...,495568,MSI,1890,6117.64,5934.11,5043.99,6117.64,6117.64,5199.99,15,5,9,1,12,2024-04-25 03:36:29.337716
4,placa-de-video-rtx-4070-ti-super-gaming-verto-...,RTX 4070 TI,https://www.kabum.com.br//produto/520758/placa...,520758,Pny,1570,5199.99,0.00,0.00,0.00,0.00,5199.99,0,0,0,1,36,2024-04-25 03:36:29.337716


In [85]:
print(df2.isna().sum())

full_name                 0
model                     0
link                      0
sku                       0
brand_name                0
weight                    0
price                     0
primePrice                0
primePriceWithDiscount    0
oldPrice                  0
oldPrimePrice             0
priceWithDiscount         0
discountPercentage        0
rating                    0
ratingCount               0
available                 0
warranty                  0
date                      0
dtype: int64


In [86]:
df3 = df2['full_name']

df3 = df3.str.replace('Placa', '')
df3 = df3.str.replace('de Vídeo', '')
df3 = df3.str.replace('De Vídeo', '')
df3 = df3.str.replace('De Video', '')
df3 = df3.str.strip()

df_names = df3.unique()
df_names = [name[:45] for name in df_names]

In [87]:
import plotly.graph_objs as go

figs = []  # Lista para armazenar os gráficos individuais

# Iterar sobre os modelos únicos
for model in df2['model'].unique():
    fig = go.Figure()
    model_data = df2[df2['model'] == model].copy()
    model_data['date'] = model_data['date'].dt.floor('min')  # Truncar para minutos

    for product in model_data['full_name'].unique():
        product_data = model_data[model_data['full_name'] == product]
        fig.add_trace(go.Scatter(x=product_data['date'], y=product_data['price'], mode='lines+markers', name=product))

    fig.update_layout(title=f'Evolução dos Preços - Modelo: {model}',
                      xaxis_title='Data',
                      yaxis_title='Preço',
                      yaxis_tickprefix='R$ ',
                      yaxis_tickformat='.2f',
                      margin=dict(l=50, r=50, t=80, b=150),
                      legend=dict(x=10, y=0, orientation='v', font=dict(size=10), itemwidth=30))

    figs.append(fig)

# Exibir os gráficos
for i, fig in enumerate(figs, 1):
    fig.show()


In [88]:
df2.head(1)

,full_name,model,link,sku,brand_name,weight,price,primePrice,primePriceWithDiscount,oldPrice,oldPrimePrice,priceWithDiscount,discountPercentage,rating,ratingCount,available,warranty,date
0,placa-de-video-rtx-4070-ti-ventus-msi-nvidia-g...,RTX 4070 TI,https://www.kabum.com.br//produto/403373/placa...,403373,MSI,1732,6117.64,5934.11,5043.99,6117.64,6117.64,5199.99,15,4,8,1,36,2024-04-25 03:36:29.337716


In [89]:
data_inicial = df['date'].min()
df2['hour'] = (df['date'] - data_inicial).dt.total_seconds() / 3600

In [90]:
df2.head(1)

,full_name,model,link,sku,brand_name,weight,price,primePrice,primePriceWithDiscount,oldPrice,oldPrimePrice,priceWithDiscount,discountPercentage,rating,ratingCount,available,warranty,date,hour
0,placa-de-video-rtx-4070-ti-ventus-msi-nvidia-g...,RTX 4070 TI,https://www.kabum.com.br//produto/403373/placa...,403373,MSI,1732,6117.64,5934.11,5043.99,6117.64,6117.64,5199.99,15,4,8,1,36,2024-04-25 03:36:29.337716,0.0


In [16]:
# df_produtos.replace(False, 0, inplace=True)
# df_produtos.replace(True, 1, inplace=True)
# df_produtos

NameError: name 'df_produtos' is not defined

In [92]:
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import Normalizer

df_produtos = df2

df_produtos = df_produtos.drop(['full_name', 'link', 'sku', 'weight', 'date'], axis=1)

df_produtos = pd.get_dummies(df_produtos, columns=['brand_name', 'model'])

X = df_produtos.drop(['price'], axis=1)
y = df_produtos['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42)

In [94]:
from sklearn.model_selection import cross_val_score, KFold

modelo = LinearRegression()

num_folds = 5

# Criar um objeto KFold para dividir os dados em folds
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

scores = cross_val_score(modelo, X, y, cv=kf, scoring='r2')

print("R² médio:", scores.mean())
print("Desvio padrão dos R²:", scores.std())

R² médio: 0.9934413521306313
Desvio padrão dos R²: 0.003376429047315227


In [95]:
modelo_linear = LinearRegression()
modelo_linear.fit(X_train, y_train)
y_pred = modelo_linear.predict(X_test)
r2_linear = r2_score(y_test, y_pred)

modelo_decision_tree = DecisionTreeRegressor()
modelo_decision_tree.fit(X_train, y_train)
y_pred_decision_tree = modelo_decision_tree.predict(X_test)
r2_decision_tree = r2_score(y_test, y_pred_decision_tree)

print(f"Coeficiente de Determinação (R²) - Regressão Linear: {r2_linear}")
print(f"Coeficiente de Determinação (R²) - Árvore de Decisão: {r2_decision_tree}")

Coeficiente de Determinação (R²) - Regressão Linear: 0.8731868946848608
Coeficiente de Determinação (R²) - Árvore de Decisão: 0.9180977340132067


In [96]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [97]:
# Generate a random regression dataset
X, y = make_regression(n_samples=100, n_features=10, noise=0.1)

# Define the parameter grid for the grid search
param_grid = {
    'n_jobs': [1, 2, 3, 4, 5, 6, 7]
}

# Create a linear regression model
model = LinearRegression()

# Create a grid search object
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=3
)

# Fit the grid search object to the dataset
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print(f'Best parameters: {grid_search.best_params_}')
print(f'Best score: {grid_search.best_score_:.2f}')

# Predict the numerical values using the best model
y_pred = grid_search.best_estimator_.predict(X_test)

# Evaluate the performance using mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean squared error: {mse:.2f}')

Best parameters: {'n_jobs': 1}
Best score: 0.99
Mean squared error: 845854.62


In [98]:
# Create a linear regression model
model = LinearRegression()

# Fit the grid search object to the dataset
model.fit(X_train, y_train)

# Predict the numerical values using the best model
y_pred = model.predict(X_test)

# Evaluate the performance using mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean squared error: {mse:.2f}')

train_error = mean_squared_error(y_train, model.predict(X_train))
test_error = mean_squared_error(y_test, model.predict(X_test))

print(f"Erro de treinamento: {train_error:.2f}")
print(f"Erro de teste: {test_error:.2f}\n")

Mean squared error: 845854.62
Erro de treinamento: 4873.43
Erro de teste: 845854.62



In [99]:
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
from sklearn.model_selection import learning_curve

# Função para plotar a curva de aprendizado com Plotly
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    fig = go.Figure()
    fig.update_layout(title=title, xaxis_title="Training examples", yaxis_title="Score", yaxis=dict(range=ylim))
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    fig.add_trace(go.Scatter(x=train_sizes, y=train_scores_mean, mode='lines+markers', name='Training score'))
    fig.add_trace(go.Scatter(x=train_sizes, y=test_scores_mean, mode='lines+markers', name='Cross-validation score'))

    fig.add_trace(go.Scatter(
        x=np.concatenate([train_sizes, train_sizes[::-1]]),
        y=np.concatenate([train_scores_mean - train_scores_std,
                          (train_scores_mean + train_scores_std)[::-1]]),
        fill='toself',
        fillcolor='rgba(255, 0, 0, 0.1)',
        line=dict(color='rgba(255, 0, 0, 0)'),
        name='Training score ± 1 std'
    ))

    fig.add_trace(go.Scatter(
        x=np.concatenate([train_sizes, train_sizes[::-1]]),
        y=np.concatenate([test_scores_mean - test_scores_std,
                          (test_scores_mean + test_scores_std)[::-1]]),
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.1)',
        line=dict(color='rgba(0, 255, 0, 0)'),
        name='Cross-validation score ± 1 std'
    ))

    fig.show()

# Plotar o gráfico de dispersão
fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test, y=y_pred, mode='markers', marker=dict(color='blue'), name='Predicted vs. Actual'))
fig.add_trace(go.Scatter(x=[min(y_test), max(y_test)], y=[min(y_test), max(y_test)], mode='lines', line=dict(color='black', dash='dash'), name='Ideal Line'))
fig.update_layout(title='Predicted vs. Actual', xaxis_title='Actual', yaxis_title='Predicted', legend=dict(x=0, y=1))
fig.show()

# Plotar a curva de aprendizado
title = "Learning Curves"
plot_learning_curve(model, title, X_train, y_train, ylim=(0.0, 1.01), cv=None, n_jobs=8)